# [UDACITY] Seattle Airbnb Analysis
The Settle Airbnb Open Dateset is chosen for **UDACITY Project - Write a Data Science Blog Post**.

6 steps of CRISP-DM (Cross-Industry Standard Process for Data Mining):
1. Business Understanding
1. Data Understanding
1. Data Preparation
1. Modelling
1. Evaluation
1. Deployment



# Business Understanding & Data Understanding

Airbnb has been allowing visitors and hosts to expand their travel options and provide a more distinctive, personalised way of experiencing the world since 2008. Airbnb has evolved into a one-of-a-kind service that is used and recognised all over the world. The company's data analysis of millions of listings offered by Airbnb is a critical aspect. These millions of listings generate a lot of data, which can be analysed and used for a variety of purposes, including security, business decisions, understanding customer and provider (host) behaviour and performance on the platform, guiding marketing initiatives, and implementing innovative additional services, among others.

The **Seatlle Airbnb Open** Dataset catalogues homestay listings in Seattle, Washington from 01-2016 to 01-2017.


There are 3 subsets in the dataset:
1. The pricing for each listing id for a certain day is described in **calendar.csv**.
1. **listings.csv** contains a detailed description of each listing id, as well as review scores.
1. For each individual listing id, **reviews.csv** gives detailed reviews.

In this project, we are going to use **calendar.csv** and **listings.csv** to answer 3 business questions:
1. Which is the most busiest time in Settle?
1. Where is the most crowdest in Settle?
1. Price Prediction.

First, let's import the data & revelant libraries needed for the analysis.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
calendar = pd.read_csv('../input/seattle/calendar.csv')
listings = pd.read_csv('../input/seattle/listings.csv')
reviews = pd.read_csv('../input/seattle/reviews.csv')

## 1. Airbnb Busiest Times in Settle

Having a glance at the dataset. 
There are 4 columns. 

The **listing_id** and **price** columns are currently NOT in the correct datatype. We need to convert the data type of **listing_id** column into string and **price**'s into float.

We are going to extract **month** and **month_year** from the **date** column also.

In [ ]:
calendar.info()

In [ ]:
# listing_id
calendar['listing_id'] = calendar['listing_id'].astype(str)

# month & month_year
calendar['month'] = pd.DatetimeIndex(calendar['date']).month
calendar['year'] = pd.DatetimeIndex(calendar['date']).year
calendar['month_year'] = pd.to_datetime(calendar['date']).dt.to_period('M')

# price
calendar['price'] = calendar['price'].apply(lambda x: float((x[1:].replace(',',''))) if type(x) != float else x)

calendar.head()

An Airbnb host can configure their listing's calendar to only be open for a few days or weeks every year.
Other listings are available throughout the year (except for when it is already booked).

The price is available only when available = t.

In [ ]:
check_f = calendar[(calendar['available'] == 'f') & (calendar['price'].notnull())].shape[0]
print('The number of rows where available = f and price is present is: {}  row(s)'.format(check_f))

Only choose available = t where the price is available for all listing_id.
The calendar.csv subset is now ready. Let's pivot and plot some charts accordingly.

In [ ]:
pivot_1 = pd.pivot_table(calendar[calendar['available'] == 't'], index='month', values=['price'], aggfunc={ 'price':np.mean})
pivot_1['month'] = pivot_1.index

pivot_2 = pd.pivot_table(calendar[calendar['available'] == 't'], index='month_year', values=['price'], aggfunc={'price':np.mean})
pivot_2['month_year'] = pivot_2.index
pivot_2['month_year'] = [str(x) for x in pivot_2['month_year']]

pivot_3 = pd.pivot_table(calendar, index='month', values=['listing_id'], columns=['available'], aggfunc={'listing_id': 'count'})
pivot_3['month'] = pivot_3.index
pivot_3['available_t'] = pivot_3.iloc[:,1]
pivot_3['available_f'] = pivot_3.iloc[:,0]

pivot_3['availibility_rate'] = pivot_3['available_t']/(pivot_3['available_t']+pivot_3['available_f'])
pivot_3 = pivot_3[['month','availibility_rate']].reset_index(drop=True)
pivot_3

plt.figure(figsize=(30, 10))
g = plt.GridSpec(2, 2) #Grid


plt.subplot(g[0,0])
plt.bar(pivot_3['month'], pivot_3['availibility_rate'])
plt.xlabel('month')
plt.xticks(pivot_3['month'])
plt.ylabel('availibility_rate')
plt.title('availibility_rate by month')


plt.subplot(g[0,1])
plt.bar(pivot_1['month'], pivot_1['price'])
plt.xlabel('month')
plt.xticks(pivot_1['month'])
plt.ylabel('avg_price')
plt.title('Avg price by month')

plt.subplot(g[1,:])
plt.bar(pivot_2['month_year'], pivot_2['price'])
plt.xlabel('month_year')
plt.xticks(pivot_2['month_year'])
plt.ylabel('avg_price')
plt.title('Avg price by month_year')

plt.show()

Generally, the busiest times of the year is in the summer (the peak in July) when the price and the listingid_count are highest.

## 2. Where in Settle has the most Airbnb properties?

We are going to use listings.csv to answer the 2nd question. 

In [ ]:
zip = listings['zipcode'].value_counts()
zip = zip.to_frame().reset_index()
zip = zip.rename({'index':'zipcode', 'zipcode':'listingid_count'}, axis=1)
zip['zipcode'].astype('str')


plt.figure(figsize = (20,10))
plt.bar(zip['zipcode'], zip['listingid_count'])
plt.xlabel('zipcode')
plt.xticks(zip['zipcode'])
plt.ylabel('listingid_count')
plt.title('listingid_count by zipcode')
plt.show()

In [ ]:
print('Top 5 areas which have the highest number of Airbnb properties in Seattle are: ')
zip.head()

# 3. Price Prediction

listings - Detailed listings data for each listing, including 92 attributes. For price prediction, only relevant features are chosen:
**'price','accommodates','bathrooms','bedrooms','beds','weekly_price','monthly_price','cleaning_fee', 'instant_bookable','reviews_per_month','cancellation_policy'**

In [ ]:
print('The listings.csv subset has {} columns and {} rows.'.format(listings.shape[1], listings.shape[0]))

In [ ]:
relevant_features = ['price','accommodates','bathrooms','bedrooms','beds','weekly_price','monthly_price','cleaning_fee', 'instant_bookable','reviews_per_month','cancellation_policy']
listings = listings[relevant_features]
listings.head()

### Data Preparation
* String into Float for price-related columns
* Missing values in numberic variables
* Missing values in categorical variables

#### String into Float for price-related columns

In [ ]:
listings['price'] = listings['price'].apply(lambda x: float((x[1:].replace(',',''))) if type(x) != float else x)
listings['weekly_price'] = listings['weekly_price'].apply(lambda x: float((x[1:].replace(',',''))) if type(x) != float else x)
listings['monthly_price'] = listings['monthly_price'].apply(lambda x: float((x[1:].replace(',',''))) if type(x) != float else x)
listings['cleaning_fee'] = listings['cleaning_fee'].apply(lambda x: float((x[1:].replace(',',''))) if type(x) != float else x)


The correlation coefficient is a number that ranges from -1 to 1.
* A number near 0 indicates a lower association (exact 0 implying no correlation).
* Closer to 1, the stronger the favourable association.
* A negative correlation is stronger if the value is closer to -1.

In [ ]:
print('Correlation among numeric variables: \n')

plt.figure(figsize=(14,10))
corr = listings.corr()
print(corr['price'], '\n')

print('Present the result in heatmap: ')
sns.heatmap(corr, annot=True)
plt.show()

listings sub-dataset still contains NaN values. In order to build a model, you must handle them. Otherwise,the fit model will break.

In [ ]:
listings.info()

#### Missing values in numberic variables

Imputing objects into the matrix rather than dropping them is one method to start making predictions on these values.

Drop the rows with missing response (price) values, then use the column's mean to estimate the values for all the other missing values.

In [ ]:
num_vars = listings.select_dtypes(include=['int','float']).columns
for var in num_vars:
    listings[var] = listings[var].fillna(listings[var].mean())
listings[num_vars].info()

#### Missing values in categorical variables

NaN values are frequently informative, but by leaving them as 0 in every column of your model, you are not capturing them. As a result, utilise **get dummies** to encode **NaN** values as their own dummy coded column using the **dummy na** option for categorical variables.

In [ ]:
cat_bool_vars = listings.select_dtypes(include=['object','bool']).columns
cat_bool_vars
for var in cat_bool_vars:
    listings = pd.concat([listings.drop(var, axis=1), pd.get_dummies(listings[var], prefix=var, prefix_sep='_', drop_first=True)],
                         axis=1)

y = listings['price']
X = listings.drop(columns = ['price'], inplace=False)
X.info(verbose=True)


## Modelling

4 steps to build a model:
* Instantiate your linear model using normalized data
* Fit your model on the training data
* Predict using the test data
* Score the model on the test

The fit method in sklearn breaks due to NAN values.
To assure the model extends well to new data, we must creat a train test split.
To assure the same train and test split will occure for different users, random_state is set with a fixed number.



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
lm_model = LinearRegression(normalize = True)
lm_model.fit(X_train, y_train)
y_test_predict = lm_model.predict(X_test)
y_train_predict = lm_model.predict(X_train)
test_score = r2_score(y_test, y_test_predict)
train_score = r2_score(y_train, y_train_predict)

print('test_score: ',test_score)
print('train_score: ', train_score)
if abs(test_score/train_score) > 0.8:
    print('The model is not overfitted.')
else:
    print('Need to improve the model.')
print('The r-squared score for the model is {} on {} values.'.format(test_score, len(y_test)),'\n')


coef_df = pd.DataFrame()
coef_df['feature'] = X_train.columns
coef_df['coef'] = lm_model.coef_
coef_df['abs_coef'] = np.abs(lm_model.coef_)
coef_df.sort_values(by=['abs_coef'], ascending=False)
print('Top 20 featutes which have the most impact on price: \n', coef_df, '\n')


# Summary
The business goes well in the Summer in general. Top 5 areas which have the highest number of Airbnb properties in Seattle are: 98122, 98103, 98102, 98105, 98109. Among the chosen features, accommodates has the most impact on the price.